# Install all required libraries

In [ ]:
# streamlit: For building the web application interface
# pyngrok: To expose the Streamlit app from Google Colab to the web
!pip install streamlit pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


# Import libraries

In [ ]:
import numpy as np
import scipy.stats

# The Core Simulation Function

Next, we'll write the most important function: the one that simulates a single path of the surplus process. This function will be the heart of our application.

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import scipy.stats
import plotly.graph_objects as go
import pandas as pd
import plotly.colors
import time

# --- PAGE CONFIGURATION ---
st.set_page_config(
    page_title="Ruin Theory Simulator",
    page_icon="📉",
    layout="wide"
)

# --- BACKEND SIMULATION ENGINE ---
# The simulate_one_path function remains the same as the last optimized version
def simulate_one_path(initial_surplus, interest_rate, time_horizon, premium_params, claim_params, store_path_history=False):
    """
    Optimized version: Simulates a single path. Only stores the full path if requested.
    """
    current_time = 0.0
    current_surplus = initial_surplus

    path_data = [(current_time, current_surplus)] if store_path_history else None

    lambda_premium = premium_params['rate']
    lambda_claim = claim_params['rate']
    lambda_total = lambda_premium + lambda_claim

    while current_time < time_horizon:
        waiting_time = np.random.exponential(scale=1/lambda_total)

        if current_time + waiting_time > time_horizon:
            current_surplus *= np.exp(interest_rate * (time_horizon - current_time))
            current_time = time_horizon
            if store_path_history:
                path_data.append((current_time, current_surplus))
            break

        current_surplus *= np.exp(interest_rate * waiting_time)
        current_time += waiting_time

        if store_path_history:
            path_data.append((current_time, current_surplus))

        event_type = np.random.choice(['premium', 'claim'], p=[lambda_premium/lambda_total, lambda_claim/lambda_total])

        if event_type == 'premium':
            current_surplus += np.random.choice(premium_params['values'], p=premium_params['probs'])
        else: # claim
            dist_name = claim_params['dist'].lower()
            dist_params = claim_params['params']

            if dist_name == 'pareto':
                current_surplus -= scipy.stats.pareto.rvs(b=dist_params['shape'], scale=dist_params['scale'], size=1)[0]
            elif dist_name == 'lognormal':
                current_surplus -= scipy.stats.lognorm.rvs(s=dist_params['sigma'], scale=np.exp(dist_params['mu']), size=1)[0]
            elif dist_name == 'gamma':
                current_surplus -= scipy.stats.gamma.rvs(a=dist_params['shape'], scale=dist_params['scale'], size=1)[0]
            else:
                raise ValueError(f"Unsupported distribution: {dist_name}")

        if store_path_history:
            path_data.append((current_time, current_surplus))

        if current_surplus <= 0:
            return {'is_ruined': True, 'path_data': path_data, 'ruin_time': current_time}

    return {'is_ruined': False, 'path_data': path_data, 'ruin_time': None}


# --- STREAMLIT USER INTERFACE ---

# Sidebar remains the same
st.sidebar.title("⚙️ Simulation Parameters")
with st.sidebar.expander("General Settings", expanded=True):
    p_initial_surplus = st.number_input("Initial Surplus (u)", min_value=0.0, value=100.0, step=10.0)
    p_time_horizon = st.number_input("Time Horizon (τ)", min_value=1.0, value=50.0, step=5.0)
    p_interest_rate = st.slider("Risk-Free Interest Rate (γ)", min_value=0.0, max_value=0.2, value=0.03, step=0.005, format="%.3f")
    p_num_paths = st.number_input("Number of Simulations (N)", min_value=100, max_value=1000000000, value=10000, step=1000)
with st.sidebar.expander("Premium Distribution", expanded=True):
    p_lambda_premium = st.number_input("Premium Arrival Rate (λ₁)", min_value=0.1, value=5.0, step=0.1)
    st.markdown("###### Premium Amount (Discrete Distribution)")
    col1, col2 = st.columns(2)
    with col1:
        p_prem_val1 = st.number_input("Value 1", value=10.0, step=1.0)
        p_prem_val2 = st.number_input("Value 2", value=15.0, step=1.0)
        p_prem_val3 = st.number_input("Value 3", value=20.0, step=1.0)
    with col2:
        p_prem_prob1 = st.number_input("Prob 1", min_value=0.0, max_value=1.0, value=0.5, step=0.05)
        p_prem_prob2 = st.number_input("Prob 2", min_value=0.0, max_value=1.0, value=0.3, step=0.05)
        p_prem_prob3 = st.number_input("Prob 3", min_value=0.0, max_value=1.0, value=0.2, step=0.05)
with st.sidebar.expander("Claim Distribution", expanded=True):
    p_lambda_claim = st.number_input("Claim Arrival Rate (λ₂)", min_value=0.1, value=2.0, step=0.1)
    p_claim_dist = st.selectbox("Claim Severity Distribution", ['pareto', 'lognormal', 'gamma'])
    claim_dist_params = {}
    if p_claim_dist == 'pareto':
        st.markdown("###### Pareto Parameters")
        p_pareto_shape = st.number_input("Shape (α)", min_value=1.01, value=1.5, step=0.1)
        p_pareto_scale = st.number_input("Scale (xm)", min_value=0.1, value=30.0, step=1.0)
        claim_dist_params = {'shape': p_pareto_shape, 'scale': p_pareto_scale}
    elif p_claim_dist == 'lognormal':
        st.markdown("###### Lognormal Parameters")
        p_lognorm_mu = st.number_input("Log Mean (μ)", value=3.0, step=0.1)
        p_lognorm_sigma = st.number_input("Log Std Dev (σ)", min_value=0.1, value=0.8, step=0.1)
        claim_dist_params = {'mu': p_lognorm_mu, 'sigma': p_lognorm_sigma}
    elif p_claim_dist == 'gamma':
        st.markdown("###### Gamma Parameters")
        p_gamma_shape = st.number_input("Shape (k)", min_value=0.1, value=2.0, step=0.1)
        p_gamma_scale = st.number_input("Scale (θ)", min_value=0.1, value=25.0, step=1.0)
        claim_dist_params = {'shape': p_gamma_shape, 'scale': p_gamma_scale}

st.title("📉 Ruin Probability Monte Carlo Simulator")
st.markdown("Use the sidebar to configure parameters and click the button below to run the analysis.")

if st.button("🚀 Run Simulation", type="primary", use_container_width=True):
    p_prem_probs = [p_prem_prob1, p_prem_prob2, p_prem_prob3]
    if not np.isclose(sum(p_prem_probs), 1.0):
        st.sidebar.error("Error: Premium probabilities must sum to 1.0")
    else:
        # --- REFACTORED SIMULATION LOGIC WITH PROGRESS BAR ---

        # 1. Package parameters
        premium_params = {'rate': p_lambda_premium, 'values': [p_prem_val1, p_prem_val2, p_prem_val3], 'probs': p_prem_probs}
        claim_params = {'rate': p_lambda_claim, 'dist': p_claim_dist, 'params': claim_dist_params}

        # 2. Initialize result aggregators
        ruined_path_count = 0
        ruin_times = []
        sample_paths_data = []
        num_paths_to_plot = 5

        # 3. Create progress bar and status text placeholders
        st.markdown("---")
        progress_bar = st.progress(0)
        status_text = st.empty()

        # 4. The main simulation loop is now here in the UI script
        start_time = time.time()
        for i in range(p_num_paths):
            should_store_history = i < num_paths_to_plot

            result = simulate_one_path(
                p_initial_surplus, p_interest_rate, p_time_horizon,
                premium_params, claim_params,
                store_path_history=should_store_history
            )

            if result['is_ruined']:
                ruined_path_count += 1
                ruin_times.append(result['ruin_time'])

            if should_store_history:
                sample_paths_data.append(result)

            # 5. Update progress bar and status text
            progress_percentage = (i + 1) / p_num_paths
            progress_bar.progress(progress_percentage)
            status_text.text(f"Simulating Path: {i + 1}/{p_num_paths}")

        end_time = time.time()

        # Clear the progress bar and status text after completion
        progress_bar.empty()
        status_text.success(f"Simulation completed in {end_time - start_time:.2f} seconds.")

        # 6. Finalize and store results
        ruin_probability = ruined_path_count / p_num_paths if p_num_paths > 0 else 0
        average_ruin_time = np.mean(ruin_times) if ruin_times else 0

        st.session_state.results = {
            'ruin_probability': ruin_probability, 'average_ruin_time': average_ruin_time,
            'sample_paths': sample_paths_data, 'total_paths': p_num_paths,
            'ruined_paths': ruined_path_count
        }
        st.session_state.params_used = {"Initial Surplus": p_initial_surplus, "Time Horizon": p_time_horizon, "Interest Rate": p_interest_rate, "Num Simulations": p_num_paths, "Premium Params": premium_params, "Claim Params": claim_params}


# Display results (this part remains the same)
if 'results' in st.session_state:
    results = st.session_state.results
    st.header("📊 Simulation Results")
    col1, col2, col3 = st.columns(3)
    col1.metric("Ruin Probability", f"{results['ruin_probability']:.2%}")
    col2.metric("Ruined Paths", f"{results['ruined_paths']:,} / {results['total_paths']:,}")
    col3.metric("Avg. Time to Ruin", f"{results['average_ruin_time']:.2f} yrs" if results['average_ruin_time'] > 0 else "N/A")
    st.subheader("Sample Surplus Paths")

    fig = go.Figure()
    fig.add_hline(y=0, line_width=2, line_dash="dash", line_color="black", opacity=0.7)

    colors = plotly.colors.qualitative.Plotly
    for i, path_result in enumerate(results['sample_paths']):
        path_data = path_result['path_data']
        time_points, surplus_points = zip(*path_data)
        line_color = colors[i % len(colors)]
        fig.add_trace(go.Scatter(x=time_points, y=surplus_points, mode='lines', name=f'Path {i+1} ({"Ruined" if path_result["is_ruined"] else "Survived"})', line=dict(width=2, color=line_color)))

    fig.update_layout(title_text="Visualization of First 5 Simulated Surplus Paths", xaxis_title="Time (Years)", yaxis_title="Surplus (u)", hovermode="x unified", legend_title="Path Status")
    st.plotly_chart(fig, use_container_width=True)

    # with st.expander("Show Simulation Parameters Used"):
    #     st.json(st.session_state.params_used)

    with st.expander("⬇️ Download Results"):
          df_data = {**st.session_state.params_used, **results}
          cols_to_drop = ['premium_params', 'claim_params', 'ruin_times', 'sample_paths']
          df = pd.DataFrame([df_data])
          df = df.drop(columns=cols_to_drop, errors='ignore')
          csv = df.to_csv(index=False).encode('utf-8')

          st.download_button(
               label="Download Summary as CSV",
               data=csv,
               file_name="single_scenario_results.csv",
               mime="text/csv",
               use_container_width=True
           )


Overwriting app.py


# How to Run the Demo (in Google Colab)

This project includes a script to quickly launch an online demo of the Streamlit application directly within the Google Colab environment. The script uses ngrok to create a temporary public URL to access the app.

The following code is responsible for launching the server:

In [ ]:
# 1. Kill any existing processes on the port
!kill -9 $(lsof -t -i:8501) >/dev/null 2>&1
!pkill ngrok >/dev/null 2>&1 #<-- Added this line to kill old ngrok tunnels
!rm -f nohup.out

# 2. Run Streamlit in the background
!nohup streamlit run app.py --server.port 8501 --server.headless True > nohup.out 2>&1 &

# 3. Wait for Streamlit to start up
import time
time.sleep(10)

# 4. Get the public URL from ngrok
from pyngrok import ngrok
from google.colab import userdata

ngrok_token = userdata.get('NGROK_AUTH_TOKEN')

ngrok.set_auth_token(ngrok_token)

# Check for existing tunnels before creating a new one
tunnels = ngrok.get_tunnels()
if tunnels:
    public_url = tunnels[0].public_url
else:
    public_url = ngrok.connect(addr='8501', proto='http')

print(f"Streamlit app is running at: {public_url}")

print("\n--- Streamlit Logs (nohup.out) ---")
!cat nohup.out

Streamlit app is running at: NgrokTunnel: "https://bf6d-34-145-39-6.ngrok-free.app" -> "http://localhost:8501"

--- Streamlit Logs (nohup.out) ---



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.39.6:8501

